In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import glob
import warnings 
warnings.filterwarnings('ignore')
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
options = webdriver.ChromeOptions()
# options.add_argument(r"C:\Users\Asif-LFD\AppData\Local\Google\Chrome\User Data\Profile 1")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# from selenium import webdriver
from selenium.webdriver.common.by import By
import pytesseract
# from PIL import Image
# import io
# import cv2
# import pytesseract
import numpy as np
import re



In [2]:
url = "https://pk.indeed.com/"
driver.get(url)
time.sleep(2)

# input keyword (input variable may be 1. job title 2. where) 
1. clear() kaam nahi kar rha dyk lyna bad me

In [3]:
var_key_word_job = "frontend"
var_key_word_where = "karachi"
var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[1]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_job)
time.sleep(2)

var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[2]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_where)
time.sleep(2)



# click find jon button
driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/button").click()


In [4]:
print("There are "+ driver.find_element("xpath","/html/body/main/div/div[1]/div/div/div[5]/div[1]/div[4]/div/div/div[2]/span[1]").text)

There are 127 jobs


# left side me companies ki details jo ati list wise os ka code

1. div (id = 'mosaic-provider-jobcards') k andar "ul" ka tag id
2. ul k andar "li" ka tag {har li me indivdual company ka card}
3. li k andar div class "job_seen_beacon" os k andar sara data






In [5]:
var_cards_company_data = driver.find_element("xpath","//div[@id='mosaic-provider-jobcards']").find_elements("xpath","ul/li")

print(len(var_cards_company_data))
# saving all data into csv
var_csv_cols = ['job_title','company_name',	'location',	'salary_range',	'job_type',	'description',	'job_link',	'indeedApply',"hiringMultipleCandidates","posted_date"]


# makinf df having columns of var_csv_cols
df = pd.DataFrame(columns=var_csv_cols)


18


In [9]:
# clicking on each card company name
var_cards_company_data[10].click()


In [10]:
# temp
soup = BeautifulSoup(var_cards_company_data[10].get_attribute('innerHTML'), 'html.parser')
print(soup.prettify())

<div class="cardOutline tapItem dd-privacy-allow result job_70705211ffc82088 resultWithShelf sponTapItem desktop vjs-highlight css-kyg8or eu4oa1w0">
 <div class="slider_container css-8xisqv eu4oa1w0" dir="auto">
  <div class="slider_list css-kyg8or eu4oa1w0">
   <div class="slider_item css-kyg8or eu4oa1w0">
    <div class="job_seen_beacon">
     <table cellpadding="0" cellspacing="0" class="jobCard_mainContent big6_visualChanges" role="presentation">
      <tbody>
       <tr>
        <td class="resultContent">
         <div class="css-1m4cuuf e37uo190">
          <h2 class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0" tabindex="-1">
           <a aria-label="full details of Frontend Engineer" class="jcs-JobTitle css-jspxzf eu4oa1w0" data-hide-spinner="true" data-hiring-event="false" data-jk="70705211ffc82088" data-mobtk="1h3ihrii5jv6e800" href="/company/Pinnacle-Enterprise/jobs/Front-End-Developer-70705211ffc82088?fccid=d1b535218451dc50&amp;vjs=3" id="job_70705211ffc82088" role="butto

In [12]:
try:
    # find salary
    salary_range=var_cards_company_data[10].find_element("xpath","//table[@class='jobCard_mainContent big6_visualChanges']").find_element("xpath","//tr").find_element("xpath","td").find_element("xpath","div[@class='heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly']").find_element("xpath","div[@class='metadata salary-snippet-container']").text
except:
    salary_range = "undefine"

salary_range


'Rs 30,000 - Rs 35,000 a month'

In [13]:
soup = BeautifulSoup(var_cards_company_data[10].find_element("xpath","//tr[@class = 'jobCardShelf']").get_attribute('innerHTML'), 'html.parser')
# print(soup.prettify())

html = str(soup.prettify())

regex = r'td class="shelfItem ([^"]*)">'

matches = re.findall(regex, html)
next_words = [match.strip() for match in matches]

print(next_words)


['indeedApply', 'hiringMultipleCandidates']


In [27]:
# posted date
posted_date = var_cards_company_data[10].find_element("xpath","//table[@class = 'jobCardShelfContainer big6_visualChanges']").find_element("xpath","//tr[@class = 'underShelfFooter']").find_element("xpath","//span[@class='date']").text.split("\n")[-1]
posted_date


'Active 2 days ago'

#  right portion

In [97]:
# page ka right side
var_right_side = driver.find_element("xpath","//div[@class = 'jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent--embedded css-axjo09 eu4oa1w0']")
#ab yaha sy user interction(kisi button py click karna etc etc) kahtam hun gaya just data scrape karna han so we need to use bs4. 

# bs4
soup = BeautifulSoup(var_right_side.get_attribute('innerHTML'), 'html.parser')
print(soup.prettify())




<div class="jobsearch-JobComponent-embeddedHeader">
 <div class="jobsearch-HeaderContainer">
  <div class="jobsearch-InfoHeaderContainer">
   <div>
    <div class="jobsearch-JobInfoHeader-title-container">
     <h2 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded" dir="ltr" lang="en">
      <span>
       Frontend Developer MT
       <span class="css-1syol37 esbq1260">
        - job post
       </span>
      </span>
     </h2>
    </div>
    <div class="jobsearch-CompanyInfoContainer">
     <div class="jobsearch-CompanyInfoWithoutHeaderImage">
      <div>
       <div class="css-1qz04qx eu4oa1w0">
        <div class="css-1h46us2 eu4oa1w0">
         <div class="css-1cjkto6 eu4oa1w0" data-company-name="true">
          <a href="https://pk.indeed.com/cmp/Avialdo-Solutions?campaignid=mobvjcmp&amp;from=mobviewjob&amp;tk=1h3iftot1jtuk800&amp;fromjk=3de2311962e93f75" target="_blank">
           Avialdo Solutions
          </a>
         </div>
        </div>
   

### left up portion (jis me job title,comapny name, location and apply karny ka button han)


In [98]:
print(soup.find('div',class_="jobsearch-InfoHeaderContainer").prettify())
# text

<div class="jobsearch-InfoHeaderContainer">
 <div>
  <div class="jobsearch-JobInfoHeader-title-container">
   <h2 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded" dir="ltr" lang="en">
    <span>
     Frontend Developer MT
     <span class="css-1syol37 esbq1260">
      - job post
     </span>
    </span>
   </h2>
  </div>
  <div class="jobsearch-CompanyInfoContainer">
   <div class="jobsearch-CompanyInfoWithoutHeaderImage">
    <div>
     <div class="css-1qz04qx eu4oa1w0">
      <div class="css-1h46us2 eu4oa1w0">
       <div class="css-1cjkto6 eu4oa1w0" data-company-name="true">
        <a href="https://pk.indeed.com/cmp/Avialdo-Solutions?campaignid=mobvjcmp&amp;from=mobviewjob&amp;tk=1h3iftot1jtuk800&amp;fromjk=3de2311962e93f75" target="_blank">
         Avialdo Solutions
        </a>
       </div>
      </div>
      <div class="css-6z8o9s eu4oa1w0">
       <div>
        Karachi
       </div>
      </div>
      <div class="css-6z8o9s eu4oa1w0">
      

In [120]:
# job title
job_title = soup.find('h2',class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded').text.replace(" - job post","")
job_title

'Frontend Developer MT'

In [100]:
# comapny name
company_name = soup.find("div",class_="css-1h46us2 eu4oa1w0").text #comapny name
company_name

'Avialdo Solutions'

In [101]:
# location
location = soup.find("div",class_="css-6z8o9s eu4oa1w0").text #location
location


'Karachi'

In [102]:
# nechy waly code me 2 divs aa rahy. may be contry ka hun. lakin abhi pata nahi q future me kaam aa skta
soup.find_all("div",class_="css-6z8o9s eu4oa1w0")
# .text #location


[<div class="css-6z8o9s eu4oa1w0"><div>Karachi</div></div>,
 <div class="css-6z8o9s eu4oa1w0"></div>]

### left down portion (jis me job description wagera han)


In [103]:
var_soup_job_des = soup.find("div",class_='jobsearch-embeddedBody css-1omm75o eu4oa1w0')

var_soup_job_des

<div class="jobsearch-embeddedBody css-1omm75o eu4oa1w0" tabindex="-1"><div class="jobsearch-BodyContainer"><div></div><div class="jobsearch-JobComponent-description css-kyg8or eu4oa1w0" tabindex="0"><div class="mosaic mosaic-empty-zone" id="aboveFullJobDescription"></div><div class="mosaic mosaic-empty-zone" id="aboveExtractedJobDescription"></div><div class="css-804pn3 eu4oa1w0" id="jobDetailsSection"><div class="jobsearch-JobDescriptionSection-title css-4m8ia3 eu4oa1w0"><h2 class="css-ji0ia1 e1tiznh50" id="jobDetails" tabindex="-1">Job details</h2><div class="css-7c0b8 eu4oa1w0"><span>No matching <a aria-label="job preferences (opens in a new window)" class="css-6pji5a emf9s7v0" href="https://profile.indeed.com/preferences" rel="noopener" target="_blank">job preferences<svg aria-hidden="true" class="css-r5jz5s eac13zx0" fill="currentColor" focusable="false" role="img" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M14.504 3a.5.5 0 00-.5.5v1a.5.5 0 00.5.5h3.085l-9.59

In [104]:
# job type
job_type = var_soup_job_des.find("div",class_="css-tvvxwd ecydgvn1").text
job_type


'Full-time'

In [105]:
# jobDescriptionText
description = var_soup_job_des.find("div",id="jobDescriptionText").text
# save in text file
with open("jobDescriptionText.txt","w",encoding="utf-8") as f:
    f.write(var_soup_job_des.find("div",id="jobDescriptionText").text)
    f.close()
description

"A junior front-end developer MT will be responsible for developing and maintaining the visual elements of a website or application. This includes designing and implementing user interfaces, ensuring that the website is visually appealing and easy to navigate, and ensuring that the website is responsive across different devices.\nResponsibilities\n\nDevelop and maintain the user interface for our machine translation system.\nCollaborate with backend developers to integrate frontend components.\nCreate responsive designs using HTML, CSS, and Bootstrap.\nWrite efficient code using JavaScript and jQuery.\nTest and fix frontend issues.\n\nQualifications\n\nBachelor's degree in Computer Science or related field.\nStrong proficiency in HTML, CSS, Bootstrap, JS, and jQuery.\nExperience with version control systems (e.g., Git).\nStrong problem-solving skills and attention to detail.\nExcellent communication and teamwork abilities.\nFamiliarity with machine translation technologies is a plus.\n

In [106]:
# getting link
job_link = driver.current_url
job_link

'https://pk.indeed.com/jobs?q=frontend&l=karachi&from=searchOnHP&vjk=3de2311962e93f75'

In [114]:

# making an empty dictionary on above listt
var_dict = {i:'' for i in var_csv_cols}
var_dict

{'job_title': '',
 'company_name': '',
 'location': '',
 'salary_range': '',
 'job_type': '',
 'description': '',
 'job_link': '',
 'indeedApply': '',
 'hiringMultipleCandidates': ''}

In [116]:
# if next_words have any of var_dict keys than put 1 of each keys values in var_dic
for i in next_words:
    if i in var_dict.keys():
        var_dict[i] = 1
    else:
        var_dict[i] = 0

var_dict

{'job_title': '',
 'company_name': '',
 'location': '',
 'salary_range': '',
 'job_type': '',
 'description': '',
 'job_link': '',
 'indeedApply': 1,
 'hiringMultipleCandidates': 1}

In [117]:
var_dict["job_title"] =job_title
var_dict["company_name"] =company_name
var_dict["location"] =location
var_dict["job_type"] =job_type
var_dict["description"] =description
var_dict["job_link"] =job_link
print(var_dict)


{'job_title': 'Frontend Developer MT - job post', 'company_name': 'Avialdo Solutions', 'location': 'Karachi', 'salary_range': '', 'job_type': 'Full-time', 'description': "A junior front-end developer MT will be responsible for developing and maintaining the visual elements of a website or application. This includes designing and implementing user interfaces, ensuring that the website is visually appealing and easy to navigate, and ensuring that the website is responsive across different devices.\nResponsibilities\n\nDevelop and maintain the user interface for our machine translation system.\nCollaborate with backend developers to integrate frontend components.\nCreate responsive designs using HTML, CSS, and Bootstrap.\nWrite efficient code using JavaScript and jQuery.\nTest and fix frontend issues.\n\nQualifications\n\nBachelor's degree in Computer Science or related field.\nStrong proficiency in HTML, CSS, Bootstrap, JS, and jQuery.\nExperience with version control systems (e.g., Git)

In [118]:
# adding var_dict to df
df = df.append(var_dict,ignore_index=True)
df

,job_title,company_name,location,salary_range,job_type,description,job_link,indeedApply,hiringMultipleCandidates
0,Frontend Developer MT - job post,Avialdo Solutions,Karachi,,Full-time,A junior front-end developer MT will be respon...,https://pk.indeed.com/jobs?q=frontend&l=karach...,1,1
